# Sumatra export

In [195]:
import json
import pandas
import numpy as np

!smt export
with open('.smt/records_export.json') as ff:
    data = json.load(ff)
    
with open('record1.json', 'w') as record1:
    for entry in range(len(data)):
        record1.write(json.dumps(data[entry], sort_keys=True, indent=4, separators=(',', ': ')))

df = pandas.DataFrame(data)  #df is now the sumatra dataframe

# Extractor() : 

This method finds the simulations which have a single ".mpz.npz" file in their Data/ folder and gets all the information out of that file into a dataframe

In [176]:
import glob
import os
import fipy as fp

# print df['output_data'][0][-2]['path']
# df['datastore'][0]['parameters']['root']  #alternative method to get data directory

labels = []

labels = df['label']
print labels

dfC = pandas.DataFrame(columns=['label','f','steps','time', 'N', 'cvar', 'cvarInterp', 'x', 'y'])

def extractor(labelz, Lx=200):  #will work as long as there is 1 .mpz.npz file in the simulation directory
    rows = 0
    for lbl in labelz:
        dictt = {} #create a dictionary to fill in with simulation data
        dictt.update({'label':lbl})
        datapath = os.path.join('/data/aem1/new1a/surf-research/Andrey/Data', lbl)
        mfile = glob.glob('{0}/*.mpz.npz'.format(datapath)) #get the name of the mpz file with data we need
        
        if len(mfile)==1:
            fn = np.load(mfile[0])
            for item in fn:
                dictt.update({str(item):fn[item]}) #add every variable from the file into the dictionary
            dictt.update({'cvarInterp':None}) #create a blank column for later
            
            #now we need the x and y coordinates for interpolation to match the grid size
            N = dictt['N']
            m = fp.Grid2D(nx = N, ny= N, dx=Lx / N, dy=Lx / N)
            dictt.update({'x':m.x})
            dictt.update({'y':m.y})
            dfC.loc[rows]=dictt        

    return dfC #dfC is a compact dataframe with all the information we need
        
extractor(labels)

1a10x1step30.npz
0    48841eea4a91
1    da25c86b72f0
2    bf22b1bfaef6
3    7f52183544f2
4    47235abe3a7b
Name: label, dtype: object


,label,f,steps,time,N,cvar,cvarInterp,x,y
0,7f52183544f2,0.786513270533,32,1.0,10,"[0.518911886971, 0.521427016706, 0.52280435362...",None,"[10.0, 30.0, 50.0, 70.0, 90.0, 110.0, 130.0, 1...","[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10...."


# cvar Interpolation

The method below will take the datafram which is created by the extractor() above and save the interpolation function into the dataframe

In [208]:
from scipy import interpolate as scpinter    

def cvarInterp(cvarFrame):
    for variables in dfC.iterrows():
        x = variables[1]['x'] #get the variables needed for interpolation
        y = variables[1]['y']
        cvar = variables[1]['cvar']
        function = scpinter.interp2d(x, y, cvar, copy=False)
        variables[1]['cvarInterp'] = function #set the interpolated function into the dataframe column
        
cvarInterp(dfC)

# cvar Figures of Merit

Now we have everything we need in the dataframe and can compare the different grid sizes, calculate L2 using cvar etc

In [ ]:
cvar_times = []
def CvarCompare(dframe, params.json, cvar_times):
    return True

# Free Energy Interpolation

For this section, we have to take the elapsed time and cvar from each step file, calculate the energy at that step, then interpolate the array of energies along with time. Save the intepolated function into the SUMATRA dataframe

In [223]:
energyFunctions = []
energyFunctions.append(None)
print energyFunctions
print ' '
for row in df.iterrows():
    print row[1]['label']
    
df

#How to select a specific row and then you can change the values in it's columns
df.loc[df['label']=='bf22b1bfaef6']['datastore']

[None]
 
48841eea4a91
da25c86b72f0
bf22b1bfaef6
7f52183544f2
47235abe3a7b


2    {u'type': u'FileSystemDataStore', u'parameters...
Name: datastore, dtype: object

In [ ]:
import os
import numpy as np
from scipy import interpolate as scpinter

results = {}
c_alpha = 0.3
c_beta = 0.7
kappa = 2.0
# M = 5.0
# c_0 = 0.5
# epsilon = 0.01
rho_s = 5.0

def f_0(c):
    return rho_s*((c - c_alpha)**2)*((c_beta-c)**2)

def f(c):
    return (f_0(c)+ .5*kappa*(c.grad.mag)**2)


def freeEnergyInterp(labels, dframe, Lx = 200):
    energyFunctions = []
    #This code will access every step file saved in the Data/[label] directory 
    for label in dframe['label']:
        e_elaps = []
        e_enrg = []
        filepath = os.path.join('/data/aem1/new1a/surf-research/Andrey/Data', label)
        #gets the last step file in the directory (hopefully at equillibrium)
        file = os.listdir('Data/{0}'.format(label))[len(os.listdir('Data/{0}'.format(label)))-2] 
        stepfiles = glob.glob('{0}/*0.npz'.format(datapath)) #get the list of step files for simulation
        if len(stepfiles)==0: 
            energyFunctions.append(None)
        else:
            for stpfile in stepfiles:
                fn = np.load(stpfile)
                #do free enrgy math here


                cells = stpfile['N']**2
                dx = Lx / fn['N']
                f(c_var).cellVolumeAverage*cells*(dx**2)


                e_elaps.append[fn['elapsed']]
                e_enrg.append[energy]

            eFunction = scpinter.interp1d(e_elaps, e_enrg copy=False)
            energyFunctions.append(eFunction) #save a list of all the energy functions
            
        dFrame['Free_Energy_Function'] = energyFunctions #add the list as a new column to the dataframe


#             data = {'cvar':fn['cvar'], 'freeEn':float(fn['f']), 'step':int(fn['steps']), 'elapsed':float(fn['time'])}
#     #         print type(data)
#     #         print type(str(fn['grid']))
#             results.update({str(fn['grid']):data})


# Figures of Merit

Now we need to use methods to add some figures of merit to the dataframes to compare the grids to the 400 grid

In [ ]:
for row in df.iterrows():

In [219]:
#add moose data as well
#Add a method to check for equal parameters between runs of different grid sizes

ideal_times = []
for i in range(160): ideal_times.append(i)
    
def FreeEnergyCompare(dframe, ideal_times, params.json):
    L1Norms = [] #should these be dictionaries to match with the proper simulation label? 
    L2Norms = []
    LinfNorms = []
    
    #Need to put in checks to make sure all the parameters are the same for the simulation between different gird sizes
    #there could be a table of 20 simulations and we need to make sure the simulations get grouped properly
    # maybe add a json file parameter to each function? check if the df['params'] matches with the json file
    for row in dframe.iterrows():
        if row[1]['N']==400 and ParamsCheck(dfram, params.json)==True:
            energyFunct400 = row[1]['Free_Energy_Function']
    for row in dframe.iterrows():    
        if ParamsCheck(dfram, params.json)==True:
            efunction = row[1]['Free_Energy_Function']
            diff = np.absolute(efunction[ideal_times] - energyFunct400[ideal_times])
            L1Norms.append(np.linalg.norm(diff,1)) #maybe add as dictionary with a matching label
            L2Norms.append(np.linalg.norm(diff,2)) #maybe add as dictionary with a matching label
            LinfNorms.append(np.linalg.norm(diff, np.inf)) #maybe add as dictionary with a matching label
    
    
    
    #Anushkas code
#     diff_50.append(np.absolute(y50x4[ideal_times] - y400x5[ideal_times]))
#     L1Norms.append(np.linalg.norm(diff_50,1))
#     L2Norms.append(np.linalg.norm(diff_50,2))
#     LinfNorms.append(np.linalg.norm(diff_50, np.inf))
    

IndentationError: expected an indented block (<ipython-input-219-2a4d61f10dc9>, line 4)

In [36]:
results['10x1']['freeEn']

0.7865168606271677

In [30]:
print len(df['output_data'][0])

4


In [31]:
for p in 'N', 'steps', 'sweeps':
    df[p] = df.parameters.map(lambda x: json.loads(x['content'])[p])

In [ ]:
print df['parameters'][0]['content']

In [37]:
df['Free Energy']=results['10x1']['freeEn']
df['elapsed']=results['10x1']['elapsed']

columns.append('Free Energy')
columns.append('elapsed')
df[columns]

,label,N,steps,tags,duration,sweeps,memory,Free Energy,elapsed
0,8f1f49c60b96,10,20,[npzTest],4.042712,2,4648.0,0.786517,0.57275


In [33]:
columns = ['label', 'N', 'steps', 'tags', 'duration', 'sweeps']
print df[columns]

          label   N  steps       tags  duration  sweeps
0  8f1f49c60b96  10     20  [npzTest]  4.042712       2


In [34]:
import numpy as np
import os
num = 0
memoryArray = np.array([])
for num in range(len(df.index)):
    memName = df['output_data'][num][len(df['output_data'][num])-1]['path']
    memlocation = df['datastore'][num]['parameters']['root']
    pool = os.path.join(memlocation, memName)
    memdata = np.loadtxt(pool)
    memoryArray = np.append(memoryArray, memdata)
    print memoryArray
    print ' '
    print num

[ 4648.]
 
0


In [35]:
df['memory']=memoryArray
columns.append('memory')
df[columns]

,label,N,steps,tags,duration,sweeps,memory
0,8f1f49c60b96,10,20,[npzTest],4.042712,2,4648.0


In [38]:
data = {'a' : [1, 2, 3], 'b' : [4, 5, 6]}
pandas.DataFrame(data)

,a,b
0,1,4
1,2,5
2,3,6


In [59]:
data = [{'a' : 1, 'b' : 4}, {'a' : 2, 'b' : 5}, {'a' : 3 ,'b' : 6}]
aa  = pandas.DataFrame(data)

In [60]:
def recalc(row):
    return row.b * 2

In [61]:
print aa

   a  b
0  1  4
1  2  5
2  3  6


In [ ]:
aa['c'] = [recalc(row) for index, row in aa.iterrows()]
print aa

In [73]:
times = np.array([0., 1.0, 1.5])
values = times**2


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(times, values)

In [ ]:
import numpy as np
np.linspace(0., 100., 1000)

In [83]:
!ls Data/1de38ff12ec7/
import glob
glob.glob('Data/1de38ff12ec7/*.npz')

1a10x1step0.npz  1a10x1step10.npz  1a10x1step20.npz  memory_usage.txt


['Data/1de38ff12ec7/1a10x1step0.npz',
 'Data/1de38ff12ec7/1a10x1step10.npz',
 'Data/1de38ff12ec7/1a10x1step20.npz']